In [1]:
# Imports
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import SVD, SVDpp, NMF
from surprise import SlopeOne, CoClustering
from surprise.prediction_algorithms import predictions

%load_ext autoreload
%autoreload 2

In [2]:
from helpersSlopeOne import*

# Load dataset and samples into a pandas data frame
DATA_TRAIN_PATH = 'data/data_train.csv'
data_np = load_data(DATA_TRAIN_PATH)


DATA_TEST_PATH = 'data/sampleSubmission.csv'
samples = load_data(DATA_TEST_PATH)

In [4]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(data_np[['user_id', 'movie_id', 'rating']], reader=reader)

## Basic Slope One
ID : 25592
- RMSE : 1.040
- Secondary: 0.134
- RMSE computed : 1.0004

In [5]:
slope_cv = cross_validate(SlopeOne(), data, cv=5, n_jobs=5, verbose=False)

In [6]:
print('SlopeOne', '\t', round(slope_cv['test_rmse'].mean(), 4), '\t', round(slope_cv['test_mae'].mean(), 4))

SlopeOne 	 1.0003 	 0.8018


In [4]:
trainset = data.build_full_trainset()
slpone_algo = SlopeOne().fit(trainset)

## Report RMSE

- RMSE : 1.000192760925377

In [7]:
from sklearn.model_selection import train_test_split

train_data_np, test_data_np = train_test_split(data_np[['user_id', 'movie_id', 'rating']], test_size = 0.2 ,random_state=1)

train_data = Dataset.load_from_df(train_data_np[['user_id', 'movie_id', 'rating']], reader=reader)
test_data = Dataset.load_from_df(train_data_np[['user_id', 'movie_id', 'rating']], reader=reader)

In [16]:
trainset = train_data.build_full_trainset()

slpone_algo = SlopeOne().fit(trainset)

test_data_np['ratingpred'] = test_data_np[['user_id', 'movie_id']] \
.apply(lambda row: slpone_algo.predict(row['user_id'], row['movie_id'])[3], axis=1)

In [21]:
test_data_np['ratingpredround'] = np.round(test_data_np['ratingpred'].values)

In [23]:
RMSE = ((test_data_np.ratingpred - test_data_np.rating) ** 2).mean() ** .5
print (RMSE)

1.000192760925377
